In [30]:
import bs4
import requests
import re
from typing import Dict, Any, List
import json
import random

In [3]:
data_url = "https://pokemondb.net/evolution/level"

In [5]:
# get html from the URL
response = requests.get(data_url)

In [7]:
# get the table with id "evolution"
soup = bs4.BeautifulSoup(response.text, 'html.parser')
table = soup.find('table', {'id': 'evolution'})

In [9]:
rows = table.find_all('tr')

In [15]:
rows[1]

<tr>
<td class="cell-fixed cell-name"><picture class="infocard-cell-img">
<source height="56" srcset="https://img.pokemondb.net/sprites/scarlet-violet/icon/avif/bulbasaur.avif" type="image/avif" width="60"/>
<img alt="Bulbasaur" class="img-fixed icon-pkmn" height="56" loading="lazy" src="https://img.pokemondb.net/sprites/scarlet-violet/icon/bulbasaur.png" width="60"/>
</picture><span class="infocard-cell-data"><a class="ent-name" href="/pokedex/bulbasaur" title="View Pokedex for #0001 Bulbasaur">Bulbasaur</a></span>
</td>
<td class="text-center">→</td>
<td class="cell-fixed cell-name"><picture class="infocard-cell-img">
<source height="56" srcset="https://img.pokemondb.net/sprites/scarlet-violet/icon/avif/ivysaur.avif" type="image/avif" width="60"/>
<img alt="Ivysaur" class="img-fixed icon-pkmn" height="56" loading="lazy" src="https://img.pokemondb.net/sprites/scarlet-violet/icon/ivysaur.png" width="60"/>
</picture><span class="infocard-cell-data"><a class="ent-name" href="/pokedex/ivy

In [65]:
def extract_poke_level_data(rows: list) -> dict:
    poke_level_data = {}
    for row in rows[1:]:
        poke_name = row.find_all('td')[2].find('img')['src'].split('/')[-1].split('.')[0]
        min_level = int(row.find_all('td')[3].text.strip())
        devolve_target = row.find_all('td')[0].find('img')['src'].split('/')[-1].split('.')[0]
        poke_level_data[poke_name] = {
            'min_level': min_level,
            'devolve_target': devolve_target.capitalize().replace('-', ' ')
        }
    return poke_level_data
def extract_trainer_data(key: str, text: str) -> Dict[str, Any]:
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    
    trainer_data = {'key': key, 'team': []}
    current_pokemon = None
    
    for line in lines:
        if ':' in line and not line.startswith('-'):
            # Key-value pair; could be trainer metadata or part of a Pokémon
            k, v = line.split(':', 1)
            k = k.strip()
            v = v.strip()
            if current_pokemon is None:
                # Trainer-level metadata
                trainer_data[k.lower().replace(" ", "_")] = v
            else:
                if k.lower() == 'level':
                    current_pokemon['level'] = int(v)
                elif k.lower() == 'ivs':
                    
                    current_pokemon['ivs'] = v
                else:
                    # Arbitrary key-value at Pokémon level
                    current_pokemon[k.lower().replace(" ", "_")] = v
        elif line.startswith('-'):
            # Move line
            if current_pokemon is not None:
                current_pokemon.setdefault('moves', []).append(line[1:].strip())
        else:
            # New Pokémon name (line with no colon or dash)
            if current_pokemon:
                trainer_data['team'].append(current_pokemon)
            current_pokemon = {'name': line.strip()}
    
    if current_pokemon:
        trainer_data['team'].append(current_pokemon)
    
    return trainer_data
def output_trainer_data(trainer: object) -> str:
    t_data = f"=== {trainer['key']} ===\n"
    for key, value in trainer.items():
        if key != "team" and key != "key":
            if key == "ai":
                fixed_key = "AI"
            elif key == "double_battle":
                fixed_key = "Double Battle"
            else:
                fixed_key = key.replace('_', ' ').capitalize()
            t_data += f"{fixed_key}: {value}\n"
        
    t_data += "\n"
    if "team" in trainer:
        for p in trainer["team"]:
            t_data += f"{p['name']}\n"
            t_data += f"Level: {p.get('level', 'N/A')}\n"
            if "ivs" in p:
                t_data += f"IVs: {p['ivs']}\n"
            if "moves" in p:
                m_count = 0
                for m in p["moves"]:
                    if m_count >= 4:
                        break
                    t_data += f"- {m.title()}\n"
                    m_count += 1
                
            t_data += "\n"
    return t_data.strip()


In [52]:
min_levels = extract_poke_level_data(rows)

In [61]:
def should_devolve(pokemon: str, level: int) -> bool:
    if pokemon.lower() in min_levels:
        return level < min_levels[pokemon.lower()]["min_level"]
    return False

In [53]:
min_levels["nidorina"]

{'min_level': 16, 'devolve_target': 'Nidoran f'}

In [33]:
filepath = "C:\\projects\\pokefirered-expansion\\src\\data\\trainers.party"
with open(filepath, 'r', encoding="utf8") as file:
    trainers_data = file.read()
trainers = {}

pattern = r"=== (.*?) ===\n(.*?)(?=\n=== |\n$)"
trainer_blobs = re.findall(pattern, trainers_data, re.DOTALL)

for t in trainer_blobs:
    trainers[t[0]] = extract_trainer_data(t[0], t[1])

In [64]:
for k, v in trainers.items():
    if v['team']:
        for p in v['team']:
            if should_devolve(p["name"].lower(), p["level"]):
                print(f"Devolving {p['name']} for trainer {k} at level {p['level']}")
                p["name"] = min_levels[p["name"].lower()]["devolve_target"]

Devolving Pidgeotto for trainer TRAINER_YOUNGSTER_BEN at level 13
Devolving Pidgeotto for trainer TRAINER_YOUNGSTER_JOEY at level 16
Devolving Pidgeotto for trainer TRAINER_YOUNGSTER_DILLON at level 17
Devolving Pidgeotto for trainer TRAINER_YOUNGSTER_BEN_2 at level 17
Devolving Kakuna for trainer TRAINER_BUG_CATCHER_RICK at level 6
Devolving Pidgeotto for trainer TRAINER_LASS_SALLY at level 9
Devolving Pidgeotto for trainer TRAINER_LASS_2 at level 15
Devolving Whirlipede for trainer TRAINER_TEAM_ROCKET_GRUNT_9 at level 19
Devolving Whirlipede for trainer TRAINER_TEAM_ROCKET_GRUNT_12 at level 21
Devolving Whirlipede for trainer TRAINER_TEAM_ROCKET_GRUNT_18 at level 20
Devolving Metapod for trainer TRAINER_BUG_CATCHER_5 at level 6
Devolving Kakuna for trainer TRAINER_BUG_CATCHER_5 at level 3
Devolving Whirlipede for trainer TRAINER_BIKER_2 at level 7
Devolving Kakuna for trainer TRAINER_BUG_CATCHER_ANTHONY at level 4


In [66]:
trainer_parties = ""
for t_name, t in trainers.items():
    trainer_parties += output_trainer_data(t) + "\n\n"

In [67]:
with open(filepath, 'w', encoding='utf-8') as file:
    file.write(trainer_parties)